In [1]:
print("HelloWorld")

HelloWorld


In [2]:
from langchain_community.document_loaders import UnstructuredPDFLoader
path = "./PDF.pdf"
loader = UnstructuredPDFLoader(path)

In [4]:
data = loader.load()

c:\Users\khatt\Documents\Model-Context-Protocol\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
# print(data[0].page_content)

MARKDOWN_SEPARATORS = [
    "\n#{1,6} ",
    "```\n",
    "\n\\*\\*\\*+\n",
    "\n---+\n",
    "\n___+\n",
    "\n\n",
    "\n",
    " ",
    "",
]

In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    separators=MARKDOWN_SEPARATORS,
    chunk_size=1000,
    chunk_overlap=200
)

In [19]:
processdata =splitter.split_documents(data)

In [20]:
processdata

[Document(metadata={'source': './PDF.pdf'}, page_content='Google Cloud’s AI Adoption Framework\n\nContents\n\nCreating value through AI, every step of the way\n\nPart 1: Executive summary\n\nThe power of AI ................................................................................................................... 4\n\nLeveraging the power of AI ............................................................................................. 5\n\nThe AI maturity themes\n\nThe AI maturity phases\n\nThe AI Maturity Scale\n\nPutting it all together\n\nNext steps ........................................................................................................................... 12\n\nFind out more\n\nWork with Google experts\n\nPart 2: Technical deep-dive\n\nThe AI maturity phases ................................................................................................... 14\n\nTactical\n\nStrategic\n\nTransformational\n\nThe AI Maturity Scale .............................

In [8]:
from langchain_community.embeddings import HuggingFaceEmbeddings
model_name = "thenlper/gte-small"

In [9]:
embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    multi_process=True,
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity
)

C:\Users\khatt\AppData\Local\Temp\ipykernel_21808\3142973650.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


In [23]:
len(embedding_model.embed_query("HEY I AM TESTING THIS MODEL for fun"))

384

In [5]:
from tqdm.notebook import tqdm
from pinecone import Pinecone
import os
import sys
from dotenv import load_dotenv
load_dotenv()

YOUR_PINECONE_KEY=os.getenv('pinecone')
pc = Pinecone(api_key=YOUR_PINECONE_KEY)


In [34]:
index = pc.Index("vectorized")

In [36]:
data = []
for i, entry in enumerate(processdata[:10]):
    text = entry.page_content
    vector = embedding_model.embed_query(text)
    data.append(
        {
            "id": "vec{}".format(i),
            "values": vector,
            "metadata": {"text": text}
        }
    )

In [41]:
index.upsert(
    vectors=data,
    namespace= "GoogleData"
)

{'upserted_count': 10}

In [44]:
user_input = input("User: ")

vectorized_input = embedding_model.embed_query(user_input)
context = index.query(
    namespace="GoogleData",
    vector=vectorized_input,
    top_k=1,
    include_metadata=True
)

In [22]:
import os
from pinecone import Pinecone
from dotenv import load_dotenv
from langchain_community.embeddings import HuggingFaceEmbeddings
# envpath = "./database/.env"
# load_dotenv(envpath)
load_dotenv()
def google(user_input:str)->str:
    """ When user ask about GOOGLE CLOUD AI FRAME WORK """

    
    pc = Pinecone(api_key=PINECONE)
    index = pc.Index("vectorized")

    model_name = "thenlper/gte-small"
    embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    multi_process=True,
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity
)
    
    vectorized_input = embedding_model.embed_query(user_input)
    context = index.query(
    namespace="GoogleData",
    vector=vectorized_input,
    top_k=1,
    include_metadata=True
)
    return context['matches'][0]['metadata']['text']

In [23]:
google("What is Ai")

C:\Users\khatt\AppData\Local\Temp\ipykernel_17952\501838731.py:16: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


'Machine learning truly is reshaping the marketplace.\n\n3 Artificial intelligence is the theory and development of systems able to perform tasks normally requiring human intelligence, such as visual perception, speech recognition, and decision-making. Machine learning is an effective way for building AI systems through automatically discovering useful patterns from data, rather than feeding human-writ- ten rules to the system. 4 Machine Learning: The New Proving Ground for Competitive Advantage. 5 Notes from the AI frontier: Modeling the impact of AI on the world economy, McKinsey & Company, September 2018.\n\n4\n\nLeveraging the power of AI\n\nHow do you structure your teams for success? How can you create, discover, share, and\n\nmanage data assets? How can you leverage native cloud technologies to scale AI? How do\n\nyou streamline the process of updating and monitoring your ML models in production?\n\nWe have a solution for that.\n\nThe AI maturity themes'